In [4]:
from scipy.fft import fft
import numpy as np
import matplotlib.pyplot as plt
import glob
import torch 
import librosa
from os import listdir
import re
import audioread

In [5]:
dict_classes = {
    0: 'portuguese',
    1: 'china',
    2: 'french',
    3: 'quebec',
    4: 'maroc',
    5: 'japon',
    6: 'american',
    7: 'english',
    8: 'korean',
    9: 'spain',
    10: 'italy',
    11: 'tunis'
    }
list_genres = ['M', 'F']

In [6]:
def trim(record, min_duration):
    half_duration = min_duration//2

    # First, we compute the barycenter of energy along time. We interpret it as the moment when the word appears
    barycenter = int(np.floor(np.multiply(np.power(record,2),np.arange(0,record.shape[0],1)).sum()/np.power(record,2).sum()))

    # Second, we adjust the barycenter to be in the index range
    if barycenter-half_duration < 0:
        barycenter += half_duration-barycenter
    if barycenter+half_duration >= record.shape[0]:
        barycenter -= barycenter+half_duration - record.shape[0]
    
    # Finally, we trim the recording around the barycenter 
    return record[barycenter-half_duration:barycenter+half_duration]

In [1]:
import os
import librosa
import numpy as np

def save_spectrograms_from_folder(input_dir, output_dir, sr=16000, n_mels=128, duration=5.0):
    """
    Parcourt un dossier contenant des fichiers audio, calcule et sauvegarde leurs spectrogrammes.
    
    Args:
        input_dir (str): Dossier contenant les fichiers audio (.wav, .mp3, etc.).
        output_dir (str): Dossier où sauvegarder les spectrogrammes.
        sr (int): Fréquence d'échantillonnage (exemple : 16000 Hz).
        n_mels (int): Nombre de bandes mel pour le spectrogramme.
        duration (float): Durée maximale de chaque audio en secondes.
    """
    # Créer le dossier de sortie si nécessaire
    os.makedirs(output_dir, exist_ok=True)

    # Lister tous les fichiers dans le dossier d'entrée
    audio_files = [f for f in os.listdir(input_dir) if f.endswith(('.wav', '.mp3'))]

    for audio_file in audio_files:
        try:
            # Chemin complet du fichier audio
            file_path = os.path.join(input_dir, audio_file)
            
            # Charger l'audio et tronquer à `duration` secondes
            y, _ = librosa.load(file_path, sr=sr, duration=duration)
            
            # Calculer le spectrogramme mel
            spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
            
            # Convertir en échelle dB
            spectrogram_db = librosa.power_to_db(spectrogram, ref=np.max)
            
            # Normalisation (optionnelle)
            spectrogram_db = (spectrogram_db - spectrogram_db.mean()) / spectrogram_db.std()
            
            # Sauvegarder en fichier .npy
            output_path = os.path.join(output_dir, audio_file.replace('.wav', '.npy').replace('.mp3', '.npy'))
            np.save(output_path, spectrogram_db)
            
            print(f"Spectrogramme sauvegardé : {output_path}")
        
        except Exception as e:
            print(f"Erreur avec le fichier {audio_file} : {e}")

# Exemple d'utilisation
input_dir = "./test"  # Dossier contenant vos fichiers audio
output_dir = "./test_spectre"  # Dossier où sauvegarder les spectrogrammes
save_spectrograms_from_folder(input_dir, output_dir)


Spectrogramme sauvegardé : ./test_spectre/english_F_49.npy
Spectrogramme sauvegardé : ./test_spectre/japon_H_46.npy
Spectrogramme sauvegardé : ./test_spectre/korean_F_48.npy
Spectrogramme sauvegardé : ./test_spectre/korean_F_50.npy
Spectrogramme sauvegardé : ./test_spectre/american_F_50.npy
Spectrogramme sauvegardé : ./test_spectre/quebec_H_47.npy
Spectrogramme sauvegardé : ./test_spectre/tunis_H_46.npy
Spectrogramme sauvegardé : ./test_spectre/english_H_49.npy
Spectrogramme sauvegardé : ./test_spectre/korean_H_49.npy
Spectrogramme sauvegardé : ./test_spectre/english_F_50.npy
Spectrogramme sauvegardé : ./test_spectre/american_H_49.npy
Spectrogramme sauvegardé : ./test_spectre/portuguese_F_47.npy
Spectrogramme sauvegardé : ./test_spectre/maroc_F_50.npy
Spectrogramme sauvegardé : ./test_spectre/italy_F_48.npy
Spectrogramme sauvegardé : ./test_spectre/china_F_50.npy
Spectrogramme sauvegardé : ./test_spectre/maroc_F_46.npy
Spectrogramme sauvegardé : ./test_spectre/french_H_48.npy
Spectrogr

In [2]:
# Exemple d'utilisation
input_dir = "./train"  # Dossier contenant vos fichiers audio
output_dir = "./train_spectre"  # Dossier où sauvegarder les spectrogrammes
save_spectrograms_from_folder(input_dir, output_dir)

Spectrogramme sauvegardé : ./train_spectre/quebec_H_27.npy
Spectrogramme sauvegardé : ./train_spectre/american_F_38.npy
Spectrogramme sauvegardé : ./train_spectre/maroc_H_12.npy
Spectrogramme sauvegardé : ./train_spectre/french_F_19.npy
Spectrogramme sauvegardé : ./train_spectre/quebec_H_37.npy
Spectrogramme sauvegardé : ./train_spectre/tunis_H_17.npy
Spectrogramme sauvegardé : ./train_spectre/china_H_29.npy
Spectrogramme sauvegardé : ./train_spectre/korean_F_40.npy
Spectrogramme sauvegardé : ./train_spectre/portuguese_H_6.npy
Spectrogramme sauvegardé : ./train_spectre/china_H_4.npy
Spectrogramme sauvegardé : ./train_spectre/china_H_37.npy
Spectrogramme sauvegardé : ./train_spectre/italy_H_7.npy
Spectrogramme sauvegardé : ./train_spectre/quebec_H_34.npy
Spectrogramme sauvegardé : ./train_spectre/japon_F_33.npy
Spectrogramme sauvegardé : ./train_spectre/spain_F_6.npy
Spectrogramme sauvegardé : ./train_spectre/english_F_21.npy
Spectrogramme sauvegardé : ./train_spectre/maroc_F_40.npy
Spe

In [3]:
# Exemple d'utilisation
input_dir = "./validation"  # Dossier contenant vos fichiers audio
output_dir = "./validation_spectre"  # Dossier où sauvegarder les spectrogrammes
save_spectrograms_from_folder(input_dir, output_dir)

Spectrogramme sauvegardé : ./validation_spectre/quebec_H_45.npy
Spectrogramme sauvegardé : ./validation_spectre/french_F_41.npy
Spectrogramme sauvegardé : ./validation_spectre/english_F_42.npy
Spectrogramme sauvegardé : ./validation_spectre/portuguese_H_42.npy
Spectrogramme sauvegardé : ./validation_spectre/italy_F_42.npy
Spectrogramme sauvegardé : ./validation_spectre/tunis_H_45.npy
Spectrogramme sauvegardé : ./validation_spectre/spain_H_43.npy
Spectrogramme sauvegardé : ./validation_spectre/italy_F_44.npy
Spectrogramme sauvegardé : ./validation_spectre/italy_H_42.npy
Spectrogramme sauvegardé : ./validation_spectre/english_H_44.npy
Spectrogramme sauvegardé : ./validation_spectre/english_H_43.npy
Spectrogramme sauvegardé : ./validation_spectre/italy_F_41.npy
Spectrogramme sauvegardé : ./validation_spectre/american_H_42.npy
Spectrogramme sauvegardé : ./validation_spectre/quebec_H_41.npy
Spectrogramme sauvegardé : ./validation_spectre/american_H_44.npy
Spectrogramme sauvegardé : ./valida